In [1]:
import pickle

# Load meta dict

with open('../../PNC/AllSubjectsMeta.bin', 'rb') as f:
    meta = pickle.load(f)
    
# Load rest subject ids and splits

with open('../../Work/Abstract/PaperBin/AllThreeSplit.bin', 'rb') as f:
    splits = pickle.load(f)
    subids = splits['allThreeYesWrat']
    groups = splits['groups']
    
print(len(subids))

593


In [2]:
import numpy as np

subidsNp = np.array(subids)

# Load timeseries

def loadSeries(prefix, para, idx):
    with open('{:}/{:}_fmri_power264/timeseries/{:}.bin'.format(prefix, para, idx), 'rb') as f:
        return pickle.load(f)

rest_ts = [loadSeries('../../PNC', 'rest', meta[subid]['rest']) for subid in subidsNp]
nback_ts = [loadSeries('../../PNC', 'nback', meta[subid]['nback']) for subid in subidsNp]
emoid_ts = [loadSeries('../../PNC', 'emoid', meta[subid]['emoid']) for subid in subidsNp]

print('Loading complete')

Loading complete


In [3]:
import numpy as np

def normalizeSubjects(subjects):
    for i in range(len(subjects)):
        subj = subjects[i]
        subj -= np.mean(subj, axis=1, keepdims=True)@np.ones([1,subj.shape[1]])
        subj /= np.std(subj, axis=1, keepdims=True)@np.ones([1,subj.shape[1]])
        if np.sum(np.isnan(subj)) > 0:
            print(i)
        if np.sum(np.isinf(subj)) > 0:
            print(i)

normalizeSubjects(rest_ts)
normalizeSubjects(nback_ts)
normalizeSubjects(emoid_ts)

print('Complete')

Complete


In [4]:
# Calculate pearson matrices

rest_p = np.stack([np.corrcoef(sub) for sub in rest_ts])
nback_p = np.stack([np.corrcoef(sub) for sub in nback_ts])
emoid_p = np.stack([np.corrcoef(sub) for sub in emoid_ts])

print(rest_p.shape)
print('Complete')

(593, 264, 264)
Complete


In [5]:
# Create feature vectors (right now just ages, maleness, and femaless)

males = 0
females = 0

X_all = []
for subid in subidsNp:
    subj = meta[subid]
    maleness = 1 if subj['meta']['Gender'] == 'M' else 0
    femaleness = 1 if maleness == 0 else 0
    feat = np.array([subj['meta']['AgeInMonths'], maleness, femaleness])
    X_all.append(feat)
    if maleness == 1:
        males += 1
    if femaleness == 1:
        females += 1
X_all = np.vstack(X_all)

print(f'{males} {females}')
print(X_all[10:20])
print('Complete')

271 322
[[223   1   0]
 [190   0   1]
 [197   0   1]
 [145   1   0]
 [148   0   1]
 [142   0   1]
 [123   1   0]
 [176   1   0]
 [129   0   1]
 [173   1   0]]
Complete


In [6]:
import torch

def convertTorch(p):
    t = torch.from_numpy(p).float()
    u = []
    for i in range(t.shape[0]):
        u.append(t[i][torch.triu_indices(264,264,offset=1).unbind()])
    return torch.stack(u).cuda()

def normalizeP(p):
    return p - torch.mean(p, dim=1, keepdim=True)

rest_p_t = convertTorch(rest_p)
nback_p_t = convertTorch(nback_p)
emoid_p_t = convertTorch(emoid_p)

# rest_p_t = normalizeP(rest_p_t)
# nback_p_t = normalizeP(nback_p_t)
# emoid_p_t = normalizeP(emoid_p_t)

print(rest_p_t.shape)
print(nback_p_t.shape)
print(emoid_p_t.shape)
print('Complete')

torch.Size([593, 34716])
torch.Size([593, 34716])
torch.Size([593, 34716])
Complete


In [7]:
wratDict = dict()

with open('../../PNC/wrat.csv', 'r') as f:
    lines = f.readlines()[1:]
    for line in lines:
        line = line.strip().split(',')
        wratDict[line[0]] = {'raw': line[2], 'std': line[3]}

wrat = []

for key in subids:
    wrat.append(float(wratDict[str(key)]['std']))
    
wrat = np.array(wrat)
wrat_t = torch.from_numpy(wrat).float().cuda()

print('Complete')

Complete


In [8]:
from math import floor
import itertools
import copy
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

def arith(n):
    return int(n*(n+1)/2)

def createCG(nPara, layerSizes, lossFn, dp=0.5, negSlope=0):
    class CG(nn.Module):
        def __init__(self):
            super(CG, self).__init__()
            self.nPara = nPara
            self.heads = []
            self.params = []
            for n in range(arith(nPara)):
                layers = []
                for i in range(len(layerSizes)-1):
                    layers.append(nn.Linear(layerSizes[i], layerSizes[i+1]).float().cuda())
                    self.params.append(layers[-1])
                layers = nn.Sequential(*layers)
                self.heads.append(layers)
            self.params = [layer.weight for layer in self.params] + [layer.bias for layer in self.params]
            self.loss = lossFn
            self.dp = nn.Dropout(p=dp)
            self.relu = nn.LeakyReLU(negative_slope=negSlope)

        def forward(self, x):
            x = self.dp(x)
            res = []
            for n in range(arith(self.nPara)):
                yA = x[:,floor(n/self.nPara),0,:]
                yB = x[:,floor(n%self.nPara),1,:]
                y = torch.cat([yA,yB],dim=1)
                for layer in self.heads[n]:
                    y = self.relu(layer(y)) if layer != self.heads[n][-1] else layer(y).squeeze()
                res.append(y)
            return torch.stack(res, dim=1)
    return CG()


def trainCG(cg, trainFeat, trainLabels, nEpochs=50, bSize=1000, pPeriod=1000, lr=2e-5, wd=2e-5, verbose=True, thresh=150):
    N = trainLabels.shape[0]
    allIdcs = torch.arange(N).long().cuda()
    pairs = list(itertools.combinations_with_replacement(np.arange(N),2))
    optim = torch.optim.Adam(cg.params, lr=lr, weight_decay=wd)

    if verbose:
        print(f'Training for {nEpochs} epochs')

    cg.train()
        
    for epoch in range(nEpochs):
        randPairs = copy.copy(pairs)
        random.shuffle(randPairs)
        nComplete = 0

        if verbose:
            print(f'epoch {epoch}')

        for n in range(0,len(randPairs),bSize):
            batchPairs = randPairs[n:n+bSize]
            Ai, Bi = zip(*batchPairs)
            Ai = torch.tensor(list(Ai)).long().cuda()
            Bi = torch.tensor(list(Bi)).long().cuda()
            A = trainFeat[Ai,:,:]
            B = trainFeat[Bi,:,:]
            a = trainLabels[Ai]
            b = trainLabels[Bi]
            optim.zero_grad()
            pos = torch.stack([A,B],dim=2)
            neg = torch.stack([B,A],dim=2)
            pres = cg(pos)
            nres = cg(neg)
#             t = pres-nres-(a-b)
#             r = pres+nres
#             tLoss = cg.loss(t, torch.zeros(t.shape).float().cuda())
#             rLoss = cg.loss(r, torch.zeros(r.shape).float().cuda())
#             (tLoss+rLoss).backward()
            pp = pres-(a-b)
            nn = nres-(b-a)
            pLoss = cg.loss(pp, torch.zeros(pp.shape).float().cuda())
            nLoss = cg.loss(nn, torch.zeros(nn.shape).float().cuda())
            rLoss = 0#1*cg.loss(pres+nres, torch.zeros(pp.shape).float().cuda())
            (pLoss+nLoss+rLoss).backward()
            optim.step()
            if n % pPeriod == 0 and verbose:
                print(f'\tpLoss={pLoss} nLoss={nLoss}')
#                 print(f'\ttLoss={tLoss} rLoss={rLoss}')
            if pLoss < thresh:
                break
        if pLoss < thresh:
            print('Early stopping')
            break
                
    if verbose:
        print(f'Completed {nEpochs*len(pairs)} comparisons') if verbose else None

def evalCG(cg, trainFeat, trainLabels, testFeat):
    N = testFeat.shape[0]
    Ntrain = trainFeat.shape[0]

#     wp = np.zeros(N)
#     wn = np.zeros(N)
#     up = np.zeros(N)
#     un = np.zeros(N)
    res = torch.zeros(N)
    
    with torch.no_grad():
        for i in range(N):
            part = testFeat[i:i+1]
            A = part.expand(trainFeat.shape)
            B = trainFeat
            b = trainLabels.unsqueeze(1)
            pos = torch.stack([A,B],dim=2)
            neg = torch.stack([B,A],dim=2)
            pdelta = cg(pos)
            ndelta = cg(neg)
            res[i] = torch.mean(pdelta-ndelta+2*b)/2
#             pres = torch.mean(pdelta + b)
#             nres = torch.mean(b - ndelta)
#             res[i] = (pres+nres)/2

#             wp[i] = pres.detach().cpu().numpy()
#             wn[i] = nres.detach().cpu().numpy()
#             up[i] = torch.std(pdelta).detach().cpu().numpy()
#             un[i] = torch.std(ndelta).detach().cpu().numpy()
        
#     return wp,wn,up,un
    return res

print('Complete')

Complete


In [9]:
para = [nback_p_t, emoid_p_t]
d = para[0].shape[1]
rmse = []

for i in range(10):
    
    trainIdcs = groups[i][0]
    testIdcs = groups[i][1]
    
    X = torch.stack(para, dim=1)
    Y = torch.from_numpy(X_all).float().cuda()
    
    gen = Y[trainIdcs][:,1:]
    wrt = wrat_t[trainIdcs].unsqueeze(1)
    age = Y[trainIdcs][:,0].unsqueeze(1)
    
    cg = createCG(len(para), [2*d,100,1], torch.nn.MSELoss(), dp=0.5, negSlope=0)
    trainCG(cg, X[trainIdcs], wrt, nEpochs=10, bSize=500, pPeriod=20000, lr=1e-4, wd=1e-4, thresh=60) 
#     wp,wn,up,un = evalCG(cg, X[trainIdcs], wrt, X[testIdcs])
    res = evalCG(cg, X[trainIdcs], wrt, X[testIdcs])
    
    rmse.append(np.mean((res.detach().cpu().numpy() - wrat_t[testIdcs].detach().cpu().numpy())**2)**0.5)
    print(f'{i} {rmse[-1]}')
    
print('Complete')

Training for 10 epochs
epoch 0
	pLoss=483.1373291015625 nLoss=482.896484375
	pLoss=390.46124267578125 nLoss=390.9689025878906
	pLoss=364.5735778808594 nLoss=365.931884765625
	pLoss=311.6099853515625 nLoss=309.0485534667969
	pLoss=244.58265686035156 nLoss=240.88641357421875
	pLoss=180.76129150390625 nLoss=180.8467254638672
	pLoss=138.0111846923828 nLoss=142.1614532470703
	pLoss=121.11915588378906 nLoss=119.94320678710938
epoch 1
	pLoss=136.9781494140625 nLoss=131.78274536132812
	pLoss=106.66012573242188 nLoss=105.58208465576172
	pLoss=99.29898071289062 nLoss=98.96348571777344
	pLoss=90.45470428466797 nLoss=97.81735229492188
	pLoss=85.58731842041016 nLoss=86.44679260253906
	pLoss=67.29454040527344 nLoss=68.74165344238281
	pLoss=70.41695404052734 nLoss=66.3091049194336
Early stopping
Completed 1423110 comparisons
0 14.296786949011645
Training for 10 epochs
epoch 0
	pLoss=478.3077392578125 nLoss=478.17791748046875
	pLoss=381.24737548828125 nLoss=380.6124267578125
	pLoss=321.21136474609375 

In [10]:
for a in rmse:
    print(a)

print(f'res {sum(rmse)/10}')

14.296786949011645
14.821192770049013
15.9138064918657
14.782493361586589
13.234105779618538
14.082729048245197
15.182124941015648
13.183582754625045
15.71625451969806
11.300121975130713
res 14.251319859084614


In [11]:
with torch.no_grad():
    D = torch.zeros(3,trainIdcs.shape[0],trainIdcs.shape[0]).float().cuda()
    for j in range(trainIdcs.shape[0]):
        A = X[j].unsqueeze(0).expand(X[trainIdcs].shape)
        B = X[trainIdcs]
        pos = torch.stack([A,B],dim=2)
        neg = torch.stack([B,A],dim=2)
        pdelta = cg(pos)
        ndelta = cg(neg)
        D[:,j,:] = (pdelta-ndelta).T.detach()

print(D[0,0:5,0:5])

tensor([[  7.7489, -59.7659,  28.6001,  30.5715,  35.3205],
        [ 46.6275,   7.2129,  66.9271,  89.7787,  84.0614],
        [-27.2935, -78.1412,  -3.0699,  25.9270,  10.2581],
        [-29.9585, -78.3382, -21.9948,  -9.0342, -21.3083],
        [-24.9221, -83.1050, -16.6878,  16.0613,   4.0183]], device='cuda:0')


In [12]:
# D0 = (D[0] + D[1] + D[2])/3
D0 = 0.5*(D[0]-D[0].T)
print(D0[0:5,0:5])

tensor([[  0.0000, -53.1967,  27.9468,  30.2650,  30.1213],
        [ 53.1967,   0.0000,  72.5342,  84.0585,  83.5832],
        [-27.9468, -72.5342,   0.0000,  23.9609,  13.4729],
        [-30.2650, -84.0585, -23.9609,   0.0000, -18.6848],
        [-30.1213, -83.5832, -13.4729,  18.6848,   0.0000]], device='cuda:0')


In [20]:
I = torch.eye(wrt.shape[0]).float().cuda()
E0 = wrt.T@I-I@wrt

In [60]:
print(E0[0:5,0:5])

tensor([[  0.,  31., -21., -21., -14.],
        [-31.,   0., -52., -52., -45.],
        [ 21.,  52.,   0.,   0.,   7.],
        [ 21.,  52.,   0.,   0.,   7.],
        [ 14.,  45.,  -7.,  -7.,   0.]], device='cuda:0')


In [91]:
X1,res,rank,sigma = torch.linalg.lstsq(X[trainIdcs,0,:].cpu(),D0.cpu(),driver='gelsy')
print(X1.shape)
print(res)
print(rank)
print(sigma)

torch.Size([34716, 534])
tensor([])
tensor(365)
tensor([])


In [92]:
A,res,rank,sigma = torch.linalg.lstsq(X[trainIdcs,0,:].cpu(),X1.T)
print(A.shape)
print(res)
print(rank)
print(sigma)

torch.Size([34716, 34716])
tensor([])
tensor(364)
tensor([])


In [98]:
from sklearn.decomposition import TruncatedSVD

np.random.seed(1)
svd = TruncatedSVD(n_components=2)
svd.fit(A)
svd.singular_values_
svd.explained_variance_ratio_

array([0.17508212, 0.0953139 ], dtype=float32)

In [99]:
MA = A@svd.components_.T
MB = svd.components_
print(MA.shape)
print(MB.shape)
print(MA)
print(MB)

torch.Size([34716, 2])
(2, 34716)
tensor([[ 3.7292e-02,  1.6202e-02],
        [ 4.8243e-05, -1.5062e-03],
        [ 2.9018e-03, -9.1920e-03],
        ...,
        [-9.7185e-04,  3.7991e-03],
        [-8.3358e-04,  3.0266e-04],
        [ 3.1793e-03,  7.2965e-03]])
[[ 0.00397236 -0.014714   -0.00755993 ... -0.01145009  0.00210089
  -0.01092923]
 [-0.03290593 -0.00287274 -0.00399866 ... -0.00285507  0.001163
  -0.00722377]]


In [100]:
MB = torch.from_numpy(MB)
XA = X[:,0,:].cpu()@MA
XB = MB@X[:,0,:].cpu().T
E = XA@XB
E = E*(torch.ones(E.shape)-torch.eye(E.shape[0]))
print(E)

tensor([[ -0.0000,  12.4801, -30.0143,  ..., -17.1752, -24.4820,   4.4301],
        [-58.2693,  -0.0000, -66.1430,  ..., -53.6112, -60.8349, -32.7653],
        [ -6.4929,  22.2627,  -0.0000,  ...,  -5.6484, -12.3976,  14.4512],
        ...,
        [  0.6876,  29.1233, -10.8730,  ...,   0.0000,  -5.7128,  21.1043],
        [-37.5083,  -7.0028, -46.5535,  ..., -34.3828,  -0.0000, -14.0213],
        [  7.9788,  36.0734,  -4.0839,  ...,   7.7609,   1.0788,   0.0000]])


In [101]:
V = torch.zeros(wrat_t.shape)
V[trainIdcs] = wrt[:,0].cpu()
V = V.unsqueeze(0)
Et = torch.mean((-E+V)[testIdcs], dim=1)
print(Et)
print(torch.mean((Et-wrat_t[testIdcs].cpu())**2)**0.5)

tensor([ 85.9637,  54.2466,  85.9395,  84.6867,  88.0491,  69.7205, 111.4181,
         99.3686,  90.8362,  88.7583,  88.6305, 104.9313, 107.4452,  86.9834,
         76.1227,  98.6855, 102.6464,  97.0766,  82.9226,  79.4212,  78.8501,
        113.1415,  95.6813, 102.7159, 101.1084,  93.0701,  94.5229,  87.0795,
         84.4448, 113.7204, 105.3719, 101.4805, 101.9571,  93.5321,  87.6187,
         92.3792,  69.3356,  99.2735,  84.5599,  90.0032, 115.8602,  94.0407,
         91.2724,  96.8521,  94.6381,  90.1724, 100.7575,  97.0485,  67.3470,
         79.3631, 104.6621,  88.1914,  70.1744,  92.0589,  93.5873,  73.2304,
        112.0574,  89.9538,  89.1707])
tensor(15.8584)


In [48]:
print(wrat_t[testIdcs].cpu())

tensor([ 89.,  89., 110.,  87.,  92.,  88.,  98., 105.,  90.,  93.,  92.,  96.,
        113., 126., 105., 110.,  80.,  94.,  85., 110.,  75., 111.,  91., 110.,
        121., 117., 117., 117.,  90., 132.,  99., 121., 128.,  98., 102.,  96.,
         72., 105.,  93.,  93., 107., 103., 100., 116.,  95., 114., 115., 117.,
         90.,  73.,  97., 105.,  89., 106., 108.,  89., 105., 109.,  85.])


In [46]:
def powerMethod(A, nIter=10, u=None):
    u = torch.rand(A.shape[1])/A.shape[1] if u is None else u
    for i in range(nIter):
        u = A@u
        u = u/torch.linalg.vector_norm(u)
        if i % 10 == 0 or i + 10 > nIter:
            print(u)
    return u

print('Complete')
# u = powerMethod(A, nIter=100)

Complete


In [78]:
a = torch.mean(A@u/u)
print(a)

tensor(0.2462)
